In [5]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, LSTMCell

import warnings
warnings.filterwarnings('ignore')

In [6]:
plt.rcParams['figure.figsize'] = (10, 7.5)
plt.rcParams['axes.grid'] = False

In [7]:
print(tf.__version__)

2.12.0


In [4]:
tf.random.set_seed(42)
np.random.seed(42)

In [8]:
train_df = pd.read_csv('./HScore/data/train.csv', index_col=0).drop(['CionScore','Coin_timestamp'],axis=1)
val_df = pd.read_csv('./HScore/data/val.csv', index_col=0).drop(['CionScore','Coin_timestamp'],axis=1)
test_df = pd.read_csv('./HScore/data/test.csv', index_col=0).drop(['CionScore','Coin_timestamp'],axis=1)

print(train_df.shape, val_df.shape, test_df.shape)

(2151, 5) (614, 5) (308, 5)


In [9]:
train_df

,Open,High,Low,Close,Volume
0,0.010909,0.009038,0.011130,0.010638,0.000274
1,0.010731,0.008742,0.010713,0.010195,0.000214
2,0.010274,0.008349,0.009911,0.009390,0.000337
3,0.009482,0.007685,0.008210,0.007804,0.000557
4,0.007877,0.006544,0.006264,0.007371,0.001044
...,...,...,...,...,...
2146,0.605723,0.589778,0.615111,0.606373,0.277508
2147,0.607224,0.611116,0.618808,0.625090,0.380599
2148,0.625683,0.609942,0.626670,0.611468,0.351140
2149,0.612158,0.594508,0.612459,0.599410,0.330366


In [14]:
class DataWindow():
    def __init__(self, input_width, label_width, shift, 
                 train_df=train_df, val_df=val_df, test_df=test_df, 
                 label_columns=None):
        
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}
        
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift
        
        self.total_window_size = input_width + shift
        
        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]
        
        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]
    
    def split_to_inputs_labels(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:,:,self.column_indices[name]] for name in self.label_columns],
                axis=-1
            )
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])
        
        return inputs, labels
    
    def plot(self, model=None, plot_col='Close', max_subplots=3):
        inputs, labels = self.sample_batch
        
        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        
        for n in range(max_n):
            plt.subplot(3, 1, n+1)
            plt.ylabel(f'{plot_col} [scaled]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                     label='Inputs', marker='.', zorder=-10)

            if self.label_columns:
              label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
              label_col_index = plot_col_index

            if label_col_index is None:
              continue

            plt.scatter(self.label_indices, labels[n, :, label_col_index],
                        edgecolors='k', marker='s', label='Labels', c='green', s=64)
            if model is not None:
              predictions = model(inputs)
              plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                          marker='X', edgecolors='k', label='Predictions',
                          c='red', s=64)

            if n == 0:
              plt.legend()

        plt.xlabel('Time (h)')
        
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32
        )
        
        ds = ds.map(self.split_to_inputs_labels)
        return ds
    
    @property
    def train(self):
        return self.make_dataset(self.train_df)
    
    @property
    def val(self):
        return self.make_dataset(self.val_df)
    
    @property
    def test(self):
        return self.make_dataset(self.test_df)
    
    @property
    def sample_batch(self):
        result = getattr(self, '_sample_batch', None)
        if result is None:
            result = next(iter(self.train))
            self._sample_batch = result
        return result

In [15]:
multi_window = DataWindow(input_width=7, label_width=7, shift=1, label_columns=['Close'])

# Chapter 14 

In [16]:
def compile_and_fit(model, window, patience=3, max_epochs=50):
    early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=patience,
                                   mode='min')
    
    model.compile(loss=MeanSquaredError(),
                  optimizer=Adam(),
                  metrics=[MeanAbsoluteError()])
    
    history = model.fit(window.train,
                       epochs=max_epochs,
                       validation_data=window.val,
                       callbacks=[early_stopping])
    
    return history

### 14.1.2 Building a mutli-step linear model 

In [20]:
ms_val_performance={}
ms_performance={}

In [24]:
ms_linear = Sequential([
    Dense(1)
])

history = compile_and_fit(ms_linear, multi_window)

ms_val_performance['Linear'] = ms_linear.evaluate(multi_window.val)
ms_performance['Linear'] = ms_linear.evaluate(multi_window.test, verbose=0)


print(f"驗證集的MSE {ms_val_performance['Linear'][1]}")
print(f"測試集的MSE {ms_performance['Linear'][1]}")

Epoch 1/50
67/67 [==============================] - 1s 3ms/step - loss: 0.0754 - mean_absolute_error: 0.1916 - val_loss: 2.2968 - val_mean_absolute_error: 1.4207
Epoch 2/50
67/67 [==============================] - 0s 2ms/step - loss: 0.0314 - mean_absolute_error: 0.1361 - val_loss: 1.1272 - val_mean_absolute_error: 0.9874
Epoch 3/50
67/67 [==============================] - 0s 2ms/step - loss: 0.0157 - mean_absolute_error: 0.1043 - val_loss: 0.5864 - val_mean_absolute_error: 0.7041
Epoch 4/50
67/67 [==============================] - 0s 2ms/step - loss: 0.0102 - mean_absolute_error: 0.0865 - val_loss: 0.3283 - val_mean_absolute_error: 0.5187
Epoch 5/50
67/67 [==============================] - 0s 2ms/step - loss: 0.0072 - mean_absolute_error: 0.0736 - val_loss: 0.1947 - val_mean_absolute_error: 0.3903
Epoch 6/50
67/67 [==============================] - 0s 2ms/step - loss: 0.0051 - mean_absolute_error: 0.0619 - val_loss: 0.1147 - val_mean_absolute_error: 0.2874
Epoch 7/50
67/67 [==========

## 14.2 Implementing a deep neural network
### 14.2.2 Implementing a deep neural network as a multi-step model 

In [23]:
ms_dense = Sequential([
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, kernel_initializer=tf.initializers.zeros),
])

history = compile_and_fit(ms_dense, multi_window)

ms_val_performance['Dense'] = ms_dense.evaluate(multi_window.val)
ms_performance['Dense'] = ms_dense.evaluate(multi_window.test, verbose=0)

Epoch 1/50
67/67 [==============================] - 1s 4ms/step - loss: 0.0241 - mean_absolute_error: 0.0995 - val_loss: 0.0749 - val_mean_absolute_error: 0.2281
Epoch 2/50
67/67 [==============================] - 0s 3ms/step - loss: 5.5304e-04 - mean_absolute_error: 0.0131 - val_loss: 0.0136 - val_mean_absolute_error: 0.0880
Epoch 3/50
67/67 [==============================] - 0s 2ms/step - loss: 2.8529e-04 - mean_absolute_error: 0.0083 - val_loss: 0.0149 - val_mean_absolute_error: 0.0927
Epoch 4/50
67/67 [==============================] - 0s 3ms/step - loss: 2.7938e-04 - mean_absolute_error: 0.0079 - val_loss: 0.0154 - val_mean_absolute_error: 0.0947
Epoch 5/50
19/19 [==============================] - 0s 1ms/step - loss: 0.0147 - mean_absolute_error: 0.0921
